# Numerical Integration Report
### M. Molter

Analytically compute the value of

$$
    I = \int^{10}_0 \sin x \, dx
$$

Then use your left-hand Riemann sum rule, the Trapezoid rule, and the Simpson 1/3 rule to compute the integral numerically using a range of step sizes $10^0 \le \Delta x \le 10^{-8}$. (It will probably be easiest if you automate this last process by using a for loop to cycle through the various values of your step size; don't forget to rest your accumulator after each integral). Make a log--log plot of the error (magnitude of the difference between the analytic and numeric answers) vs. the step size $\Delta x$. Ensure that the interval (0, 10) can be divided evenly into intervals of the step size you choose! Do the graphs look like you expect them to? How do you explain their behavior for large step sizes? For very small step sizes? How do you interpret their slopes?

### Solution

The first step, as usuall, is our library imports. `pyplot` is for plotting , and `numpy` is for heavy numerical computions.

In [2]:
import matplotlib.pyplot as plt
import numpy as np

The next step is to evaluate the integral analytically so we have an exact solution to compare to. I am lazy, so I will use `sympy` to perform the integral.

In [3]:
from sympy import Symbol, integrate

x = Symbol('x')

integrate('sin(x)', (x, 0, 10))

-cos(10) + 1

In [85]:
soln = -np.cos(10) + 1
soln

1.8390715290764525

Now we can start fleshing out a `integrate` function of our own. A lot of the code for "right-hand", "trapezoid", and "Simpson's" gets repeated, so I will stuff this all in one function with a `method=` argument.

In [74]:
def integrate(func, start, stop, step, method='trap'):
    ''' Return the numerical integration of func from start to stop. 
    
        Args:
            
            func (function):    function to be integrated
            start (float):      start of integration interval
            stop (float):       end of integration interval
            step (float):       step size for integration
            
            method (string): 
            
                'right':        right-hand Riemann sum
                'trap':         trapezoidal sum
                'simp':         Simpson's 1/3 rule
                
        Returns:
        
            float:              result of integration.
    '''
    
    assert (stop > start), "Stop must be larger than start."
    assert method in ['left', 'right', 'trap', 'simp']

    if method == 'left':
        x = np.arange(start, stop, step)
        return (func(x) * step).sum()
        
    elif method == 'right':
        x = np.arange(start + step, stop + step, step)
        return (func(x) * step).sum()
    
    elif method == 'trap':
        x = np.arange(start, stop + step, step)
        y = func(x)
        
        return (step * (0.5 * (y[:-1] + y[1:]))).sum()
        
    elif method == 'simp':
        x = np.arange(start, stop + step, step)
        y = func(x)
        
        y_left  = y[:-1]
        y_right = y[1:]
        
        x_mid = 0.5 * (x[:-1] + x[1:])
        y_mid = func(x_mid)
                
        return (y_left + 4.0 * y_mid + y_right).sum() * (step / 6.0)
    
    else:
        raise ValueError('"%s" is not a valid integration method' % method)
    

Now let's test out all three versions to make sure things are working as we'd expect.

In [101]:
func = lambda x: np.sin(x)

%timeit print(integrate(func, start=0, stop=10, step=10e-8, method='left'))
%timeit print(integrate(func, start=0, stop=10, step=10e-8, method='right'))
%timeit print(integrate(func, start=0, stop=10, step=10e-8, method='trap'))
%timeit print(integrate(func, start=0, stop=10, step=10e-8, method='simp'))

MemoryError: 

Now we need a relative error function.

In [78]:
def relative_error(numerical, exact):
    ''' Return relative error given numerical and exact solutions. '''
    
    return abs(numerical - exact) * 100 / exact

Now let's create our plots.

In [100]:
func = lambda x: np.sin(x)

dx = np.logspace(start=1, stop=-8, num=10)
plt.loglog(dx, [relative_error(integrate(func, 0, 10, i, method='left'),  soln) for i in dx], label='left')
plt.loglog(dx, [relative_error(integrate(func, 0, 10, i, method='right'), soln) for i in dx], label='right')
plt.loglog(dx, [relative_error(integrate(func, 0, 10, i, method='trap'),  soln) for i in dx], label='trapezoidal')
plt.loglog(dx, [relative_error(integrate(func, 0, 10, i, method='simp'),  soln) for i in dx], label='simpsons')
    
plt.title('Relative Error of Numerical Integration Methods')    
plt.xlabel('dx')
plt.ylabel('Relative Error (%)')
plt.legend()

plt.grid(linestyle=':', which='both') 
plt.tick_params(which='both', direction='in')
plt.xlim([10, 10e-8])    

MemoryError: 

The first thing I notice is that the `left` and `right` integration methods are produced essentially the same performance. Further, for the same number of function executions, `trapezoidal` produces far more precice results. The `simpsons` version is even better--even though it has *double* the executions its timestep would suggest. It still more than makes up for this extra step with a faster converging error.

The next major feature is that the error for Simpson's decrease to about $10^{-12}$ %, and then starts increasing again. I suspect this is the point where **roundoff** error dominates, and that the other methods would face a similar issue onces they reach similarly accurate results.